<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create fictive clinical notes from Code set (DRG + ICD)

Code set are the raw classification data, we can extract from National database (Base nationale PMSI en France). They are made of 
* classification profile made of grouping variables from DRG records which are prepared with their frequency in the national database
    - age (class)
    - sexe
    - DRG (racine GHM)
    - Main diagnosis (ICD10) : cf
    - Hospitalization management type : cf
* diagnosis associated to each classification profile, extracted with their frequencies
* procedures associated to each classification profile, specialy for surgery and technical gestures, extracted with their frequencies

From thoses raw information we produce a coded clinical scenario which will be uses a seed.

This scenario is transformed into a detail prompt that will be given to a LLM for generation.
From the combinaision of primary and related diagnosis in French discharge abstract, we derived two notions :
* Primary diagnosis : host the notion of principal pathology, it is rather the primary diagnosis of the discharge abstract or the related diagnosis when it exists and that the primary diagnosis of the discharge abstract is from the chapter "Facteurs influant sur l’état de santé" of ICD10
* The Hospitalization management type is rather the term "Primary diagnosis" or the ICD-10 code of the related diagnosis when it exists


In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
import numpy as np
import datetime as dt

In [9]:
from utils import *

In [76]:
gs = generate_scenario()
# Load official dictionaries
# col_names option allow you to algin your column names the project dictionary.
gs.load_offical_icd("cim_2024.xlsx",col_names={"code" : "icd_code","libelle":"icd_code_description"} )
gs.load_offical_procedures("ccam_actes_2024.xlsx",col_names={"code":"procedure","libelle_long":"procedure_description"} )
col_names={"Code CIM":"icd_parent_code","Localisation":"primary_site","Type Histologique":"histological_type",
           "Stade":"stage","Marqueurs Tumoraux":"biomarkers","Traitement":"treatment_recommandation","Protocole de Chimiothérapie":"chemotherapy_regimen"}
gs.load_cancer_treatement_recommandations("Tableau récapitulatif traitement cancer.xlsx",col_names ) 
col_names={"racine":"drg_parent_code","lib_spe_uma":"specialty","ratio_spe_racine":"ratio"}
gs.load_specialty_refential("dictionnaire_spe_racine.xlsx",col_names)
gs.load_referential_hospital("chu")

In [77]:
# Load data from BN  PMSI
col_names={"racine":"drg_parent_code","das": "icd_secondary_code","diag":"icd_primary_code","categ_cim":"icd_primary_parent_code",
            "mdp":"case_management_type","nb_situations":"nb","acte":"procedure",
            "mode_entree":"admission_mode",
            "mode_sortie":"discharge_disposition",
            "mode_hospit":"admission_type"}
gs.load_classification_profile("bn_pmsi_cases_20250819.csv", col_names)
gs.load_secondary_icd("bn_pmsi_related_diag_20250818.csv",col_names)
gs.load_procedures("bn_pmsi_procedures_20250818.csv",col_names)

In [78]:
#Prepare cases
# df_profile = gs.df_classification_profile.drop(columns="nb")
df_profile = gs.df_classification_profile
df_profile = df_profile[(df_profile.icd_primary_code.isin(gs.icd_codes_cancer) )  & (~df_profile.drg_parent_code.isin(gs.drg_parent_code_radio) ) ]

In [57]:
def create_system_prompt(scenario):
    if scenario['admission_type'] == "Inpatient" and scenario['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_complete.txt"
    elif scenario['admission_type'] == "Outpatient" and scenario['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_outpatient.txt"
    else:
        template_name = "scenario_onco_v1.txt"
        
    with open("templates/" + template_name, "r", encoding="utf-8") as f:
        prompt = f.read()
    
    return prompt

In [80]:
# col_names = ["icd_primary_code", "case_management_type", "drg_parent_code", "cage2","cage", "sexe", "admission_type","admission_mode", "discharge_disposition",
#              "dms", "los_mean", "los_sd", "drg_parent_description"]
# test_generation_v1 = df_profile.sample(20)[col_names].reset_index(drop=True)
test_generation_v1 = df_profile.sample(1000, weights="nb").reset_index(drop=True)
test_generation_v1

,icd_primary_code,case_management_type,drg_parent_code,age,cage,cage2,sexe,admission_type,admission_mode,discharge_disposition,...,libelle_da,gp_cas,libelle_gp_cas,ga,libelle_ga,da_gp,da_gp_ga,anseqta,aso,specialty
0,C444,Z080,09M12,ge_18,[80-[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Tissu cutané et tissu sous-cutané,X16,"Affections et traumatismes de la peau, gelures",G168,Explorations et surveillance des affections de...,D20X16,D20X16G168,2024,M,DERMATOLOGIE
1,C20,DP,06K04,ge_18,[50-60[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Digestif,K02,Endoscopies digestives et biliaires avec ou sa...,G022,Endoscopies digestives avec ou sans anesthésie,D01K02,D01K02G022,2024,M,HEPATO-GASTRO-ENTERO
2,C349,Z511,28Z07,ge_18,[70-80[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Séances,S02,Chimiothérapie pour tumeur,G190,Séances : chimiothérapie,D27S02,D27S02G190,2024,M,CANCERO ADULTE
3,C61,DP,12K06,ge_18,[60-70[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Uro-néphrologie et génital,K10,Aff. génito-urinaires avec Acte classant non o...,G209,"Séjours comprenant une biopsie prostatique, en...",D15K10,D15K10G209,2024,C,UROLOGIE
4,C50,DP,09C20,ge_18,[50-60[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Gynécologie - sein,C18,Chirurgie du sein,G107,Chirurgie pour tumeurs malignes sein,D12C18,D12C18G107,2024,C,GYNECOLOGIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,C833,DP,17C08,ge_18,[80-[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Hématologie,C21,"Chir. pour Aff. des Org. Hématopoiétiques, Lym...",G141,Chirurgie au cours des tumeurs malignes (hémat...,D16C21,D16C21G141,2024,C,CHIR.PLAST.RECONSTR.
996,C159,Z511,28Z07,ge_18,[60-70[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Séances,S02,Chimiothérapie pour tumeur,G190,Séances : chimiothérapie,D27S02,D27S02G190,2024,M,CANCERO ADULTE
997,C73,DP,10C11,ge_18,[40-50[,[18-50[,2,Outpatient,DOMICILE,DOMICILE,...,Endocrinologie,C22,"Chirurgie de la Thyroide, Parathyroide, du Tra...",G158,Chirurgie de la thyroide,D19C22,D19C22G158,2024,C,CHIRURGIE VISCERALE
998,C443,DP,09C23,ge_18,[70-80[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Tissu cutané et tissu sous-cutané,C26,Chirurgie de la peau,G218,chirurgie tumeurs malignes de la peau,D20C26,D20C26G218,2024,C,CHIR.PLAST.RECONSTR.


In [59]:
from tqdm import tqdm

In [84]:
list_scenario = []

#for i in tqdm(range(len(test_generation_v1))):
for i in tqdm(range(1)):
    profile = test_generation_v1.iloc[i].copy()
    scenario = gs.generate_scenario_from_profile(profile.drop("nb"))
    row = {k:scenario[k] for k in scenario.keys()}
    user_prompt = gs.make_prompts_marks_from_scenario(scenario)
    system_prompt = create_system_prompt(scenario)
    row["user_prompt"] = user_prompt
    row["system_prompt"] = system_prompt
    
    if  scenario["icd_primary_code"]  in gs.icd_codes_cancer  :
        prefix = """Le compte rendu suivant respecte les élements suivants :
        - les diagnostics ont une formulation moins formelle que la définition du code,
        - le type histologique et la valeur des biomarqueurs si recherchés
        - le plan du CRH est conforme aux recommandations.
        """
    else :
        prefix = """Le compte rendu suivant respecte les élements suivants :
        - les diagnostics ont une formulation moins formelle que la définition du code,
        - le type histologique et la valeur des biomarqueurs si recherchés
        - le plan du CRH est conforme aux recommandations.
        """
    
    row["prefix"] = prefix
    row["prefix_len"] = len(prefix)
    list_scenario.append(row)

100%|██████████| 1/1 [00:03<00:00,  3.19s/it]


In [95]:
keep_cols = ['age',  'cage', 'cage2','sexe', 'date_entry', 'date_discharge', 'date_of_birth',
       'first_name', 'last_name', 'icd_primary_code', 'icd_primary_description', 'icd_parent_code',
       'case_management_type','case_management_type_description', 'case_management_type_text', 
       'drg_parent_code', 'drg_parent_description',
       'icd_secondaray_code',  'text_secondary_icd_official', 
        'procedure', 'text_procedure',
        'admission_type','admission_mode', 'discharge_disposition', 'dms', 'los_mean', 'los_sd',
       'cancer_stage', 'score_TNM', 'histological_type',
       'treatment_recommandation', 'chemotherapy_regimen', 'biomarkers',
       'first_name_med', 'last_name_med',
       'department','hospital',
       'cd_md_pec', 'user_prompt', 'system_prompt','prefix','prefix_len']
df_scenario = pd.DataFrame(list_scenario)[keep_cols]
df_scenario

,age,cage,cage2,sexe,date_entry,date_discharge,date_of_birth,first_name,last_name,icd_primary_code,...,biomarkers,first_name_med,last_name_med,department,hospital,cd_md_pec,user_prompt,system_prompt,prefix,prefix_len
0,80,[80-[,[50-[,1,2024-10-03,2024-10-03,1944-08-24,Willyam,Traore,C444,...,None,Ciara,Sam mine,None,Hôpital Purpan - Centre hospitalier universita...,15,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 8...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,287


In [86]:
print(df_scenario.loc[0,"user_prompt"])

**SCÉNARIO DE DÉPART :**
- Âge du patient : 80 ans
- Sexe du patient : Masculin
- Date d'entrée : 03/10/2024
- Date de sortie : 03/10/2024
- Date de naissance : 24/08/1944
- Prénom du patient : Willyam
- Nom du patient : Traore
- Mode de prise en charge : Hospitalisation en ambulatoire pour Examen de contrôle après traitement chirurgical d'une tumeur maligne
- Codage CIM10 :
   * Diagnostic principal : Tumeur maligne de la peau du cuir chevelu et du cou (C444)
   * Diagnostic associés : 
- Sténose (de la valvule) aortique (non rhumatismale) (I350)
- Tumeur maligne secondaire et non précisée des ganglions lymphatiques de la tête, de la face et du cou (C770)
- Tumeur maligne secondaire d'autres sièges précisés (C798)
- Tumeur maligne secondaire du foie et des voies biliaires intrahépatiques (C787)
- Obésité sans précision de l'adulte avec indice de masse corporelle [IMC] égal ou supérieur à 35 kg/m² et inférieur à 40 kg/m² (E6695)

- Localisation anatomique de la tumeur primaire : Tumeur

In [23]:
df_scenario.to_csv("test_generation_v2.csv")

In [74]:
profile

icd_primary_code                                                 C821
case_management_type                                             Z511
drg_parent_code                                                 17M06
age                                                             ge_18
cage                                                          [70-80[
cage2                                                           [50-[
sexe                                                                2
admission_type                                              Inpatient
admission_mode                                               DOMICILE
discharge_disposition                                        DOMICILE
nb                                                                 90
dms                                              2,40000000000000e+00
los_mean                                                     3.228022
los_sd                                                       4.318648
drg_parent_descripti

In [227]:
# df_scenario.to_csv(gs.path_data + "test_scenario_v1.csv")

In [88]:
import os
from mistralai import Mistral

In [89]:
api_key = "j17liFFXlV5X54DDPdvax0hxHBHXuSa9"
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [97]:
resp = client.chat.complete(
     model = model,
    messages=[
        {"role": "system", "content": df_scenario.loc[0,"system_prompt"]},
        {"role": "user", "content": df_scenario.loc[0,"user_prompt"]},
        {"role": "assistant", "content": df_scenario.loc[0,"prefix"], "prefix": True},
    ],
)


TypeError: object of type 'numpy.int64' has no len()

In [99]:
print(resp.choices[0].message.content[df_scenario.loc[0,"prefix_len"] :])

- les dates sont plausibles et cohérentes.

```json
{
  "CR": "
### Motif d’hospitalisation
M. Willyam Traoré, 80 ans, est admis en hospitalisation de jour le 03/10/2024 pour un bilan postopératoire après exérèse d’un carcinome épidermoïde cutané du cuir chevelu. L’objectif est d’évaluer la réponse au traitement chirurgical et de dépister une éventuelle récidive ou progression métastatique, dans un contexte de métastases ganglionnaires cervicales et hépatiques connues.

### Antécédents
Sur le plan médical, le patient présente une sténose aortique non rhumatismale, diagnostiquée en 2020 et suivie en cardiologie. Il n’a pas d’autres antécédents cardiovasculaires notables. Une obésité morbide (IMC à 37 kg/m²) est également documentée, avec un suivi nutritionnel irrégulier. Aucun antécédent chirurgical majeur en dehors de l’exérèse cutanée récente. Pas d’allergies connues.

### Mode de vie
Ancien fumeur (30 paquets-année, sevré depuis 10 ans), sans consommation d’alcool. Retraité depuis 15

In [98]:
df_scenario.loc[0,"prefix_len"]

287